In [2]:
import boto3
s3 = boto3.resource("s3")
for buckets in s3.buckets.all():
    print buckets.name

aws-logs-310825673272-us-east-1
elasticbeanstalk-us-east-1-310825673272
hadoop-techfield-training
instacart-techfield
pro-jeff
udacity-hadoop-mapreduce


In [3]:
instacart = s3.Bucket("instacart-techfield")
for files in instacart.objects.all():
    print files.key

aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv


In [4]:
s3_client = boto3.client('s3')
open('hello.txt', 'w+b').write('Hello, world!')




In [5]:
# Upload the file to S3 keyname, bucket-name, file-name
s3_client.upload_file('hello.txt', 'instacart-techfield', 'hello-remote.txt')

In [6]:
s3_client.delete_object(Bucket='instacart-techfield', Key='hello-remote.txt')

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Fri, 27 Oct 2017 02:25:01 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'cHDvCdRhmgSMkczC3Id+n3bwFtHr5Xy+j+GlQjk32QUQ6Kojgl2DN0jnew1qRK6NvLabb6GIl0E=',
   'x-amz-request-id': 'C04F2CB03AAAAF56'},
  'HTTPStatusCode': 204,
  'HostId': 'cHDvCdRhmgSMkczC3Id+n3bwFtHr5Xy+j+GlQjk32QUQ6Kojgl2DN0jnew1qRK6NvLabb6GIl0E=',
  'RequestId': 'C04F2CB03AAAAF56',
  'RetryAttempts': 0}}

In [7]:
import findspark 
findspark.init()
from pyspark import SparkContext, SQLContext
sc = SparkContext.getOrCreate() 
sqlContext = SQLContext(sc) 



In [8]:
import boto3
s3 = boto3.client('s3')
aisles = s3.get_object(Bucket='instacart-techfield', Key='aisles.csv')

In [9]:
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(aisles['Body'].read()), encoding='utf8')


In [10]:
s_df = sqlContext.createDataFrame(df)

In [11]:
s_df.printSchema()

root
 |-- aisle_id: long (nullable = true)
 |-- aisle: string (nullable = true)



In [12]:
s_df.show(2,truncate= True)

+--------+--------------------+
|aisle_id|               aisle|
+--------+--------------------+
|       1|prepared soups sa...|
|       2|   specialty cheeses|
+--------+--------------------+
only showing top 2 rows



In [13]:
s_df.count()

134

In [14]:
len(s_df.columns), s_df.columns

(2, ['aisle_id', 'aisle'])

In [15]:
s_df.describe().show()

+-------+-----------------+--------------------+
|summary|         aisle_id|               aisle|
+-------+-----------------+--------------------+
|  count|              134|                 134|
|   mean|             67.5|                null|
| stddev|38.82653731663435|                null|
|    min|                1|air fresheners ca...|
|    max|              134|              yogurt|
+-------+-----------------+--------------------+



In [16]:
s_df.describe("aisle_id").show()

+-------+-----------------+
|summary|         aisle_id|
+-------+-----------------+
|  count|              134|
|   mean|             67.5|
| stddev|38.82653731663435|
|    min|                1|
|    max|              134|
+-------+-----------------+



In [17]:
s_df.select("aisle").show(5)

+--------------------+
|               aisle|
+--------------------+
|prepared soups sa...|
|   specialty cheeses|
| energy granola bars|
|       instant foods|
|marinades meat pr...|
+--------------------+
only showing top 5 rows



In [18]:
t1 = s_df.sample(False, 0.2, 42)
t2 = s_df.sample(True, 0.2, 41)
t1.count(),t2.count()

(20, 23)

In [19]:
s_df.select('aisle_id').rdd.map(lambda x:(x,1)).take(5)


[(Row(aisle_id=1), 1),
 (Row(aisle_id=2), 1),
 (Row(aisle_id=3), 1),
 (Row(aisle_id=4), 1),
 (Row(aisle_id=5), 1)]

In [38]:
s_df.orderBy(s_df.aisle.desc()).show(5)
# train.orderBy(train.Purchase.desc()).show(5)

+--------+--------------------+
|aisle_id|               aisle|
+--------+--------------------+
|     120|              yogurt|
|      62|         white wines|
|     115|water seltzer spa...|
|      47|vitamins supplements|
|      60|   trash bags liners|
+--------+--------------------+
only showing top 5 rows



In [42]:
s_df.withColumn('Upper_Case_Aisle', s_df.aisle).select('aisle', 'Upper_Case_Aisle').show(5)

+--------------------+--------------------+
|               aisle|    Upper_Case_Aisle|
+--------------------+--------------------+
|prepared soups sa...|prepared soups sa...|
|   specialty cheeses|   specialty cheeses|
| energy granola bars| energy granola bars|
|       instant foods|       instant foods|
|marinades meat pr...|marinades meat pr...|
+--------------------+--------------------+
only showing top 5 rows



In [49]:
s_df.registerTempTable('aisle')

In [51]:
sqlContext.sql('select * from aisle').show(5)

+--------+--------------------+
|aisle_id|               aisle|
+--------+--------------------+
|       1|prepared soups sa...|
|       2|   specialty cheeses|
|       3| energy granola bars|
|       4|       instant foods|
|       5|marinades meat pr...|
+--------+--------------------+
only showing top 5 rows

